# **Dependências**

In [1]:
!pip install mlflow --quiet

In [2]:
!pip install mlflow --quiet

In [3]:
!pip install optuna --quiet

In [4]:
!pip install pyngrok --quiet

In [5]:
!pip install ngrok --quiet

In [6]:
!pip install auto-sklearn --quiet

In [7]:
!pip install scipy==1.7.1 --quiet

In [8]:

import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn
import optuna
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats



In [9]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn import neighbors
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn import tree
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn import preprocessing

# **Dataframe**

Aqui adicionamos mais um pouco de pre-processamento, pois para utilizar algoritmos de regressão precisamos de variáveis numéricas. Sendo assim realizamos a conversão dos tipos categóricos para numéricos, e as normalizações necessárias.

In [11]:
data = pd.read_csv("https://raw.githubusercontent.com/melissafalcao/TAGDI/main/Projeto2/saida.csv?token=AHWTHUD4X4NG6S7FASU7DUTBEGYKK", encoding ="UTF-8")

In [12]:
data = data.drop(data.columns[0], axis=1)

In [13]:
data.head(5)

,Name,Score,Genders,Type,Episodes,Producers,Studios,Source,Duration,Rating,Members,Favorites,Watching,Completed,On-Hold,Dropped,Plan to Watch,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1,Finished,Estreia,GendersList,StudiosList,members_log,Score-10/1,MembersClass
0,Cowboy Bebop,0.944142,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,0.008181,Bandai Visual,Sunrise,Original,24 min. per ep.,R - 17+ (violence & profanity),0.483459,0.336956,0.119243,0.329041,0.380552,0.152699,329800.0,0.320601,0.340261,0.286693,0.205156,0.109786,0.071328,0.039222,0.030677,0.029168,0.046787,0.788462,0.778846,"['Action', 'Adventure', 'Comedy', 'Drama', 'Sc...",['Sunrise'],6.097590,2.161501,SS
1,Cowboy Bebop: Tengoku no Tobira,0.891008,"Action, Drama, Mystery, Sci-Fi, Space",Movie,0.000000,"Sunrise, Bandai Visual",Bones,Original,1 hr. 55 min.,R - 17+ (violence & profanity),0.105469,0.006383,0.004669,0.095452,0.010297,0.004407,57964.0,0.042028,0.091919,0.107826,0.074490,0.030802,0.015030,0.007098,0.004977,0.004257,0.011200,0.807692,0.807692,"['Action', 'Drama', 'Mystery', 'Sci-Fi', 'Space']",['Bones'],5.436393,1.899104,SS
2,Trigun,0.870572,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",TV,0.008181,Victor Entertainment,Madhouse,Manga,24 min. per ep.,PG-13 - Teens 13 or older,0.215824,0.070381,0.032810,0.157378,0.135511,0.079704,146918.0,0.070268,0.141336,0.187625,0.162703,0.081595,0.046764,0.024201,0.014999,0.012416,0.015763,0.778846,0.778846,"['Action', 'Sci-Fi', 'Adventure', 'Comedy', 'D...",['Madhouse'],5.747344,1.974227,SS
3,Witch Hunter Robin,0.738420,"Action, Mystery, Police, Supernatural, Drama, ...",TV,0.008181,"TV Tokyo, Bandai Visual, Dentsu, Victor Entert...",Sunrise,Original,25 min. per ep.,PG-13 - Teens 13 or older,0.036552,0.003192,0.004846,0.021152,0.027251,0.030782,33719.0,0.003051,0.008977,0.022058,0.038237,0.030292,0.023386,0.013333,0.007963,0.006425,0.003852,0.817308,0.817308,"['Action', 'Mystery', 'Police', 'Supernatural'...",['Sunrise'],4.976272,1.221583,SS
4,Bouken Ou Beet,0.698910,"Adventure, Fantasy, Shounen, Supernatural",TV,0.016688,"TV Tokyo, Dentsu",Toei Animation,Manga,23 min. per ep.,PG - Children,0.005094,0.000098,0.000724,0.003351,0.004076,0.006342,3394.0,0.000435,0.000986,0.002703,0.005635,0.005663,0.005071,0.003253,0.001855,0.001931,0.000770,0.846154,0.836538,"['Adventure', 'Fantasy', 'Shounen', 'Supernatu...",['Toei Animation'],4.121363,1.062791,S


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12421 entries, 0 to 12420
Data columns (total 34 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Name           12421 non-null  object 
 1   Score          12421 non-null  float64
 2   Genders        12421 non-null  object 
 3   Type           12421 non-null  object 
 4   Episodes       12421 non-null  float64
 5   Producers      12421 non-null  object 
 6   Studios        12421 non-null  object 
 7   Source         12421 non-null  object 
 8   Duration       12421 non-null  object 
 9   Rating         12421 non-null  object 
 10  Members        12421 non-null  float64
 11  Favorites      12421 non-null  float64
 12  Watching       12421 non-null  float64
 13  Completed      12421 non-null  float64
 14  On-Hold        12421 non-null  float64
 15  Dropped        12421 non-null  float64
 16  Plan to Watch  12421 non-null  float64
 17  Score-10       12421 non-null  float64
 18  Score-

In [15]:
data['Genders'] = data['Genders'].astype('category')
data['Type'] = data['Type'].astype('category')
data['Producers'] = data['Producers'].astype('category')
data['Duration'] = data['Duration'].astype('category')
data['Studios'] = data['Studios'].astype('category')
data['Source'] = data['Source'].astype('category')
data['Rating'] = data['Rating'].astype('category')
data['MembersClass'] = data['MembersClass'].astype('category')

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12421 entries, 0 to 12420
Data columns (total 34 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   Name           12421 non-null  object  
 1   Score          12421 non-null  float64 
 2   Genders        12421 non-null  category
 3   Type           12421 non-null  category
 4   Episodes       12421 non-null  float64 
 5   Producers      12421 non-null  category
 6   Studios        12421 non-null  category
 7   Source         12421 non-null  category
 8   Duration       12421 non-null  category
 9   Rating         12421 non-null  category
 10  Members        12421 non-null  float64 
 11  Favorites      12421 non-null  float64 
 12  Watching       12421 non-null  float64 
 13  Completed      12421 non-null  float64 
 14  On-Hold        12421 non-null  float64 
 15  Dropped        12421 non-null  float64 
 16  Plan to Watch  12421 non-null  float64 
 17  Score-10       12421 non-null  

In [17]:
le = preprocessing.LabelEncoder()
data['Genders'] = le.fit_transform(data['Genders'])
data['Source'] = le.fit_transform(data['Source'])
data['Type'] = le.fit_transform(data['Type'])
data['Producers'] = le.fit_transform(data['Producers'])
data['Duration'] = le.fit_transform(data['Duration'])
data['Studios'] = le.fit_transform(data['Studios'])
data['Rating'] = le.fit_transform(data['Rating'])
data['MembersClass'] = le.fit_transform(data['MembersClass'])

In [18]:
data.head(3)

,Name,Score,Genders,Type,Episodes,Producers,Studios,Source,Duration,Rating,Members,Favorites,Watching,Completed,On-Hold,Dropped,Plan to Watch,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1,Finished,Estreia,GendersList,StudiosList,members_log,Score-10/1,MembersClass
0,Cowboy Bebop,0.944142,28,5,0.008181,473,772,10,155,4,0.483459,0.336956,0.119243,0.329041,0.380552,0.152699,329800.0,0.320601,0.340261,0.286693,0.205156,0.109786,0.071328,0.039222,0.030677,0.029168,0.046787,0.788462,0.778846,"['Action', 'Adventure', 'Comedy', 'Drama', 'Sc...",['Sunrise'],6.097590,2.161501,5
1,Cowboy Bebop: Tengoku no Tobira,0.891008,742,0,0.000000,2704,124,10,64,4,0.105469,0.006383,0.004669,0.095452,0.010297,0.004407,57964.0,0.042028,0.091919,0.107826,0.074490,0.030802,0.015030,0.007098,0.004977,0.004257,0.011200,0.807692,0.807692,"['Action', 'Drama', 'Mystery', 'Sci-Fi', 'Space']",['Bones'],5.436393,1.899104,5
2,Trigun,0.870572,1322,5,0.008181,3322,408,6,155,3,0.215824,0.070381,0.032810,0.157378,0.135511,0.079704,146918.0,0.070268,0.141336,0.187625,0.162703,0.081595,0.046764,0.024201,0.014999,0.012416,0.015763,0.778846,0.778846,"['Action', 'Sci-Fi', 'Adventure', 'Comedy', 'D...",['Madhouse'],5.747344,1.974227,5


In [19]:
data = data.drop(columns=['Name','GendersList', 'StudiosList'])

In [20]:
data.head(3)

,Score,Genders,Type,Episodes,Producers,Studios,Source,Duration,Rating,Members,Favorites,Watching,Completed,On-Hold,Dropped,Plan to Watch,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1,Finished,Estreia,members_log,Score-10/1,MembersClass
0,0.944142,28,5,0.008181,473,772,10,155,4,0.483459,0.336956,0.119243,0.329041,0.380552,0.152699,329800.0,0.320601,0.340261,0.286693,0.205156,0.109786,0.071328,0.039222,0.030677,0.029168,0.046787,0.788462,0.778846,6.097590,2.161501,5
1,0.891008,742,0,0.000000,2704,124,10,64,4,0.105469,0.006383,0.004669,0.095452,0.010297,0.004407,57964.0,0.042028,0.091919,0.107826,0.074490,0.030802,0.015030,0.007098,0.004977,0.004257,0.011200,0.807692,0.807692,5.436393,1.899104,5
2,0.870572,1322,5,0.008181,3322,408,6,155,3,0.215824,0.070381,0.032810,0.157378,0.135511,0.079704,146918.0,0.070268,0.141336,0.187625,0.162703,0.081595,0.046764,0.024201,0.014999,0.012416,0.015763,0.778846,0.778846,5.747344,1.974227,5


In [21]:
y = data['Score']
x = data.copy()
print(y)
x = x.drop(columns=['Score','members_log','Score-10/1','MembersClass','Score-10','Score-9','Score-8','Score-7','Score-6','Score-5','Score-4','Score-3','Score-2','Score-1',])

0        0.944142
1        0.891008
2        0.870572
3        0.738420
4        0.698910
           ...   
12416    0.645777
12417    0.772480
12418    0.678474
12419    0.403270
12420    0.636240
Name: Score, Length: 12421, dtype: float64


In [22]:
x.head()

,Genders,Type,Episodes,Producers,Studios,Source,Duration,Rating,Members,Favorites,Watching,Completed,On-Hold,Dropped,Plan to Watch,Finished,Estreia
0,28,5,0.008181,473,772,10,155,4,0.483459,0.336956,0.119243,0.329041,0.380552,0.152699,329800.0,0.788462,0.778846
1,742,0,0.000000,2704,124,10,64,4,0.105469,0.006383,0.004669,0.095452,0.010297,0.004407,57964.0,0.807692,0.807692
2,1322,5,0.008181,3322,408,6,155,3,0.215824,0.070381,0.032810,0.157378,0.135511,0.079704,146918.0,0.778846,0.778846
3,1243,5,0.008181,2877,772,10,158,3,0.036552,0.003192,0.004846,0.021152,0.027251,0.030782,33719.0,0.817308,0.817308
4,1946,5,0.016688,2889,849,6,152,2,0.005094,0.000098,0.000724,0.003351,0.004076,0.006342,3394.0,0.846154,0.836538


In [23]:
x['Genders'] = (x['Genders'] - x['Genders'].min()) / (x['Genders'].max() - x['Genders'].min())
x['Type'] = (x['Type'] - x['Type'].min()) / (x['Type'].max() - x['Type'].min())
x['Producers'] = (x['Producers'] - x['Producers'].min()) / (x['Producers'].max() - x['Producers'].min())
x['Studios'] = (x['Studios'] - x['Studios'].min()) / (x['Studios'].max() - x['Studios'].min())
x['Source'] = (x['Source'] - x['Source'].min()) / (x['Source'].max() - x['Source'].min())
x['Duration'] = (x['Duration'] - x['Duration'].min()) / (x['Duration'].max() - x['Duration'].min())
x['Rating'] = (x['Rating'] - x['Rating'].min()) / (x['Rating'].max() - x['Rating'].min())
x['Plan to Watch'] = (x['Plan to Watch'] - x['Plan to Watch'].min()) / (x['Plan to Watch'].max() - x['Plan to Watch'].min())

In [24]:
data.isnull().sum()

Score            0
Genders          0
Type             0
Episodes         0
Producers        0
Studios          0
Source           0
Duration         0
Rating           0
Members          0
Favorites        0
Watching         0
Completed        0
On-Hold          0
Dropped          0
Plan to Watch    0
Score-10         0
Score-9          0
Score-8          0
Score-7          0
Score-6          0
Score-5          0
Score-4          0
Score-3          0
Score-2          0
Score-1          0
Finished         0
Estreia          0
members_log      0
Score-10/1       0
MembersClass     0
dtype: int64

In [25]:
print(x.to_numpy)

<bound method DataFrame.to_numpy of         Genders  Type  Episodes  ...  Plan to Watch  Finished   Estreia
0      0.006295   1.0  0.008181  ...       0.775025  0.788462  0.778846
1      0.166817   0.0  0.000000  ...       0.136191  0.807692  0.807692
2      0.297212   1.0  0.008181  ...       0.345240  0.778846  0.778846
3      0.279451   1.0  0.008181  ...       0.079214  0.817308  0.817308
4      0.437500   1.0  0.016688  ...       0.007948  0.846154  0.836538
...         ...   ...       ...  ...            ...       ...       ...
12416  0.477068   0.4  0.000982  ...       0.024380  1.000000  1.000000
12417  0.831610   0.2  0.000000  ...       0.000576  1.000000  1.000000
12418  0.941772   0.8  0.000000  ...       0.000679  1.000000  1.000000
12419  0.857914   0.8  0.000000  ...       0.007892  1.000000  1.000000
12420  0.604541   0.8  0.000000  ...       0.006256  1.000000  1.000000

[12421 rows x 17 columns]>


In [26]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=2020)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=2020)

In [27]:
data.head(1864)

,Score,Genders,Type,Episodes,Producers,Studios,Source,Duration,Rating,Members,Favorites,Watching,Completed,On-Hold,Dropped,Plan to Watch,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1,Finished,Estreia,members_log,Score-10/1,MembersClass
0,0.944142,28,5,0.008181,473,772,10,155,4,0.483459,0.336956,0.119243,0.329041,0.380552,0.152699,329800.0,0.320601,0.340261,0.286693,0.205156,0.109786,0.071328,0.039222,0.030677,0.029168,0.046787,0.788462,0.778846,6.097590,2.161501,5
1,0.891008,742,0,0.000000,2704,124,10,64,4,0.105469,0.006383,0.004669,0.095452,0.010297,0.004407,57964.0,0.042028,0.091919,0.107826,0.074490,0.030802,0.015030,0.007098,0.004977,0.004257,0.011200,0.807692,0.807692,5.436393,1.899104,5
2,0.870572,1322,5,0.008181,3322,408,6,155,3,0.215824,0.070381,0.032810,0.157378,0.135511,0.079704,146918.0,0.070268,0.141336,0.187625,0.162703,0.081595,0.046764,0.024201,0.014999,0.012416,0.015763,0.778846,0.778846,5.747344,1.974227,5
3,0.738420,1243,5,0.008181,2877,772,10,158,3,0.036552,0.003192,0.004846,0.021152,0.027251,0.030782,33719.0,0.003051,0.008977,0.022058,0.038237,0.030292,0.023386,0.013333,0.007963,0.006425,0.003852,0.817308,0.817308,4.976272,1.221583,5
4,0.698910,1946,5,0.016688,2889,849,6,152,2,0.005094,0.000098,0.000724,0.003351,0.004076,0.006342,3394.0,0.000435,0.000986,0.002703,0.005635,0.005663,0.005071,0.003253,0.001855,0.001931,0.000770,0.846154,0.836538,4.121363,1.062791,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1859,0.726158,87,5,0.023560,1870,860,6,158,3,0.005598,0.000359,0.001011,0.003070,0.005114,0.005512,5007.0,0.000455,0.001057,0.002561,0.006129,0.005365,0.003317,0.001331,0.000792,0.000709,0.000267,0.653846,0.634615,4.162236,1.513218,4
1860,0.719346,2351,5,0.016688,1511,737,10,155,0,0.003997,0.000652,0.001100,0.001513,0.004060,0.003062,4806.0,0.000309,0.000572,0.001311,0.002801,0.002728,0.002235,0.000715,0.000837,0.000670,0.000356,0.644231,0.634615,4.016281,1.232410,4
1861,0.598093,2353,3,0.000000,1870,737,10,26,0,0.000389,0.000000,0.000045,0.000197,0.000112,0.000286,498.0,0.000027,0.000019,0.000050,0.000197,0.000425,0.000304,0.000160,0.000204,0.000118,0.000059,0.644231,0.644231,3.016616,0.823909,1
1862,0.604905,2353,3,0.000000,1870,737,7,275,0,0.000361,0.000000,0.000037,0.000183,0.000122,0.000280,463.0,0.000029,0.000019,0.000057,0.000194,0.000361,0.000224,0.000185,0.000090,0.000118,0.000148,0.653846,0.653846,2.985875,0.564271,1


# **Métricas**

In [28]:
def eval_metrics(actual, pred):
    mse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return mse, mae, r2

#**KNN**

In [29]:
def knnreg(trial):

  with mlflow.start_run(run_name="Knn"):

    n_neighbors = trial.suggest_int('n_neighbors', 3,100)
    p = trial.suggest_int('p', 1,2)
    metric = trial.suggest_categorical('metric', ['euclidean', 'l2','manhattan','cityblock','l1', 'chebyshev','minkowski'])

    neigh = KNeighborsRegressor(n_neighbors = n_neighbors,p=p,metric=metric)
    neigh.fit(X_train, y_train)

    y_pred_val = neigh.predict(X_val)

    (mse, mae, r2) = eval_metrics(y_val, y_pred_val)
    
    mlflow.log_param("n_neighbors", n_neighbors)
    mlflow.log_param("p", p)
    mlflow.log_param("metric", metric)

    mlflow.log_metric("mse", mse)
    mlflow.log_metric("mae", mae)
    mlflow.log_metric("r2", r2)
    #mlflow.log_metric("nomemetrica", nomevariavel)
    #mlflow.log_metric("nomemetrica", nomevariavel)

    return mean_squared_error(y_val, y_pred_val)




study_knn = optuna.create_study(direction='minimize')
study_knn.optimize(knnreg,n_trials=200)

[I 2021-08-16 01:44:43,846] A new study created in memory with name: no-name-0c007e3b-bfc2-42c0-b6c6-87424a907f6d
[I 2021-08-16 01:44:44,474] Trial 0 finished with value: 0.006867490919306953 and parameters: {'n_neighbors': 32, 'p': 1, 'metric': 'manhattan'}. Best is trial 0 with value: 0.006867490919306953.
[I 2021-08-16 01:44:44,981] Trial 1 finished with value: 0.008181921182593593 and parameters: {'n_neighbors': 45, 'p': 2, 'metric': 'l2'}. Best is trial 0 with value: 0.006867490919306953.
[I 2021-08-16 01:44:45,375] Trial 2 finished with value: 0.008444466577214186 and parameters: {'n_neighbors': 80, 'p': 2, 'metric': 'minkowski'}. Best is trial 0 with value: 0.006867490919306953.
[I 2021-08-16 01:44:45,874] Trial 3 finished with value: 0.007224888673952601 and parameters: {'n_neighbors': 66, 'p': 2, 'metric': 'cityblock'}. Best is trial 0 with value: 0.006867490919306953.
[I 2021-08-16 01:44:46,260] Trial 4 finished with value: 0.008176764607436333 and parameters: {'n_neighbors':

In [30]:
trial = study_knn.best_trial
print(f'MSE: {trial.value}')
print(f'Best hyperparameters: {trial.params}')
print(trial)

MSE: 0.006688857733601972
Best hyperparameters: {'n_neighbors': 10, 'p': 1, 'metric': 'cityblock'}
FrozenTrial(number=85, values=[0.006688857733601972], datetime_start=datetime.datetime(2021, 8, 16, 1, 45, 24, 159511), datetime_complete=datetime.datetime(2021, 8, 16, 1, 45, 24, 639783), params={'n_neighbors': 10, 'p': 1, 'metric': 'cityblock'}, distributions={'n_neighbors': IntUniformDistribution(high=100, low=3, step=1), 'p': IntUniformDistribution(high=2, low=1, step=1), 'metric': CategoricalDistribution(choices=('euclidean', 'l2', 'manhattan', 'cityblock', 'l1', 'chebyshev', 'minkowski'))}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=85, state=TrialState.COMPLETE, value=None)


In [31]:
optuna.visualization.plot_optimization_history(study_knn)

In [32]:
optuna.visualization.plot_slice(study_knn)

In [33]:
optuna.visualization.plot_param_importances(study_knn)

# **Árvore de decisão**

In [34]:

def arvoredec(trial):

  with mlflow.start_run(run_name="Arvore de decisao"):

    max_depth = trial.suggest_int('max_depth',9,100,log=True)#low = none
    min_samples_leaf = trial.suggest_int('min_samples_leaf',1,10)
    max_leaf_nodes = trial.suggest_int('max_leaf_nodes',0,100)#low = none

    if(max_depth == 9):
      max_depth = None
    if(max_leaf_nodes == 0):
      max_leaf_nodes = None
    elif(max_leaf_nodes == 1):
      max_leaf_nodes +=1

    clf = tree.DecisionTreeRegressor(max_depth=max_depth,min_samples_leaf=min_samples_leaf,max_leaf_nodes=max_leaf_nodes)
    clf = clf.fit(X_train, y_train)

    y_pred_val = clf.predict(X_val)

    (mse, mae, r2) = eval_metrics(y_val, y_pred_val)
    
    mlflow.log_param("max_depth", max_depth)
    mlflow.log_param("min_samples_leaf", min_samples_leaf)
    mlflow.log_param("max_leaf_nodes", max_leaf_nodes)

    mlflow.log_metric("mse", mse)
    mlflow.log_metric("mae", mae)
    mlflow.log_metric("r2", r2)
    #mlflow.log_metric("nomemetrica", nomevariavel)
    #mlflow.log_metric("nomemetrica", nomevariavel)

    return mean_squared_error(y_val, y_pred_val)

study_decisiontree = optuna.create_study(direction='minimize')
study_decisiontree.optimize(arvoredec,n_trials=200)

[I 2021-08-16 01:46:23,985] A new study created in memory with name: no-name-d9db38a2-5a96-4345-8a0e-95a6b2a26009
[I 2021-08-16 01:46:24,082] Trial 0 finished with value: 0.006105077510555661 and parameters: {'max_depth': 95, 'min_samples_leaf': 9, 'max_leaf_nodes': 11}. Best is trial 0 with value: 0.006105077510555661.
[I 2021-08-16 01:46:24,202] Trial 1 finished with value: 0.004615220111482611 and parameters: {'max_depth': 42, 'min_samples_leaf': 10, 'max_leaf_nodes': 73}. Best is trial 1 with value: 0.004615220111482611.
[I 2021-08-16 01:46:24,325] Trial 2 finished with value: 0.004733355288067668 and parameters: {'max_depth': 10, 'min_samples_leaf': 5, 'max_leaf_nodes': 85}. Best is trial 1 with value: 0.004615220111482611.
[I 2021-08-16 01:46:24,455] Trial 3 finished with value: 0.004809605282150507 and parameters: {'max_depth': 89, 'min_samples_leaf': 5, 'max_leaf_nodes': 80}. Best is trial 1 with value: 0.004615220111482611.
[I 2021-08-16 01:46:24,575] Trial 4 finished with val

In [35]:
trial = study_decisiontree.best_trial
print(f'MSE: {trial.value}')
print(f'Best hyperparameters: {trial.params}')

MSE: 0.004530901279381071
Best hyperparameters: {'max_depth': 61, 'min_samples_leaf': 8, 'max_leaf_nodes': 89}


In [36]:
optuna.visualization.plot_optimization_history(study_decisiontree)

In [37]:
optuna.visualization.plot_slice(study_decisiontree)

In [38]:
optuna.visualization.plot_param_importances(study_decisiontree)

# **Rede Neural**

In [39]:
def mlp(trial):
  with mlflow.start_run(run_name="Rede Neural"):
    activation = trial.suggest_categorical('activation', ['identity', 'logistic', 'tanh', 'relu'])
    learning_rate_init = trial.suggest_float("learning_rate_init", 1e-5, 0.3, log=True)
    solver = trial.suggest_categorical('sovler', ['lbfgs', 'sgd', 'adam'])

    mlp_model = MLPRegressor(activation=activation,learning_rate_init=learning_rate_init,solver=solver)
    mlp_model.fit(X_train, y_train)

    y_pred_val = mlp_model.predict(X_val)

    (mse, mae, r2) = eval_metrics(y_val, y_pred_val)
    
    mlflow.log_param("activation", activation)
    mlflow.log_param("learning_rate_init", learning_rate_init)
    mlflow.log_param("solver", solver)

    mlflow.log_metric("mse", mse)
    mlflow.log_metric("mae", mae)
    mlflow.log_metric("r2", r2)
    #mlflow.log_metric("nomemetrica", nomevariavel)
    #mlflow.log_metric("nomemetrica", nomevariavel)

    return mean_squared_error(y_val, y_pred_val)


study_mlp = optuna.create_study(direction='minimize')
study_mlp.optimize(mlp,n_trials=200)

[I 2021-08-16 01:46:56,623] A new study created in memory with name: no-name-f4200d0d-696c-4a7f-ba56-59069616adef
[I 2021-08-16 01:46:57,840] Trial 0 finished with value: 0.013695509917260046 and parameters: {'activation': 'logistic', 'learning_rate_init': 0.004426463543424635, 'sovler': 'sgd'}. Best is trial 0 with value: 0.013695509917260046.
[I 2021-08-16 01:47:05,548] Trial 1 finished with value: 0.008133149339349981 and parameters: {'activation': 'tanh', 'learning_rate_init': 0.2522112971976336, 'sovler': 'lbfgs'}. Best is trial 1 with value: 0.008133149339349981.
[I 2021-08-16 01:47:07,580] Trial 2 finished with value: 0.015455278896278148 and parameters: {'activation': 'logistic', 'learning_rate_init': 0.16347845363221347, 'sovler': 'adam'}. Best is trial 1 with value: 0.008133149339349981.
[I 2021-08-16 01:47:19,271] Trial 3 finished with value: 0.007678923474873372 and parameters: {'activation': 'tanh', 'learning_rate_init': 0.0007636565222371477, 'sovler': 'lbfgs'}. Best is t

In [40]:
trial = study_mlp.best_trial
print(f'MSE: {trial.value}')
print(f'Best hyperparameters: {trial.params}')

MSE: 0.004933317418051174
Best hyperparameters: {'activation': 'relu', 'learning_rate_init': 0.006072980797803359, 'sovler': 'adam'}


In [41]:
optuna.visualization.plot_optimization_history(study_mlp)

In [42]:
optuna.visualization.plot_slice(study_mlp)

In [43]:
optuna.visualization.plot_param_importances(study_mlp)

# **Random Forest**

In [44]:
def randomforest(trial):
  with mlflow.start_run(run_name="Random Forest"):
    n_estimators = trial.suggest_int('n_estimators', 100,1000)
    max_depth = trial.suggest_int('max_depth', 2,32,log=True)
    min_samples_leaf = trial.suggest_int('min_samples_leaf',1,100)

    randforest = RandomForestRegressor(n_estimators=n_estimators,max_depth=max_depth,min_samples_leaf=min_samples_leaf)
    randforest.fit(X_train, y_train)

    y_pred_val = randforest.predict(X_val)

    (mse, mae, r2) = eval_metrics(y_val, y_pred_val)
    
    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("max_depth", max_depth)
    mlflow.log_param("min_samples_leaf", min_samples_leaf)

    mlflow.log_metric("mse", mse)
    mlflow.log_metric("mae", mae)
    mlflow.log_metric("r2", r2)
    #mlflow.log_metric("nomemetrica", nomevariavel)
    #mlflow.log_metric("nomemetrica", nomevariavel)

    return mean_squared_error(y_val, y_pred_val)

study_randomforest = optuna.create_study(direction='minimize')
study_randomforest.optimize(randomforest,n_trials=200)

[I 2021-08-16 01:58:07,104] A new study created in memory with name: no-name-209b4c0d-970f-4a18-9079-a1e1f175d65a
[I 2021-08-16 01:58:19,201] Trial 0 finished with value: 0.006686115212433441 and parameters: {'n_estimators': 886, 'max_depth': 2, 'min_samples_leaf': 47}. Best is trial 0 with value: 0.006686115212433441.
[I 2021-08-16 01:58:44,810] Trial 1 finished with value: 0.004284528263991583 and parameters: {'n_estimators': 722, 'max_depth': 7, 'min_samples_leaf': 62}. Best is trial 1 with value: 0.004284528263991583.
[I 2021-08-16 01:59:23,488] Trial 2 finished with value: 0.003292875491732628 and parameters: {'n_estimators': 688, 'max_depth': 25, 'min_samples_leaf': 10}. Best is trial 2 with value: 0.003292875491732628.
[I 2021-08-16 01:59:33,220] Trial 3 finished with value: 0.0067021287008553945 and parameters: {'n_estimators': 724, 'max_depth': 2, 'min_samples_leaf': 41}. Best is trial 2 with value: 0.003292875491732628.
[I 2021-08-16 01:59:41,287] Trial 4 finished with value:

In [45]:
trial = study_randomforest.best_trial
print(f'MSE: {trial.value}')
print(f'Best hyperparameters: {trial.params}')

MSE: 0.0030347432002446263
Best hyperparameters: {'n_estimators': 915, 'max_depth': 30, 'min_samples_leaf': 1}


In [46]:
optuna.visualization.plot_optimization_history(study_randomforest)

In [47]:
optuna.visualization.plot_slice(study_randomforest)

In [48]:
optuna.visualization.plot_param_importances(study_randomforest)

# **Melhor Resultado:**
O Modelo que obteve o menor MSE foi o random forest, com os seguintes hiper parâmetros:
{'n_estimators': 915, 'max_depth': 30, 'min_samples_leaf': 1}

In [49]:
def print_results(modelo): 
    y_pred_train = modelo.predict(X_train)
    y_pred_val = modelo.predict(X_val)
    y_pred_test = modelo.predict(X_test)
    mse_train = mean_squared_error(y_train, y_pred_train)
    mse_val = mean_squared_error(y_val, y_pred_val)
    mse_test = mean_squared_error(y_test, y_pred_test)
    print(f"MSE treino: {mse_train}")
    print(f"MSE validacao: {mse_val}")
    print(f"MSE teste: {mse_test}")

In [51]:
randomforestreg = RandomForestRegressor(n_estimators=915, max_depth=30,min_samples_leaf=1)
randomforestreg.fit(X_train, y_train)

RandomForestRegressor(max_depth=30, n_estimators=915)

In [52]:
print_results(randomforestreg)

MSE treino: 0.0004687242670972402
MSE validacao: 0.0030516863973971357
MSE teste: 0.002864553587419314


# Utilizando auto-sklearn para comparação com o modelo obtido pela nossa otimização de hiperparâmetros

In [10]:
from autosklearn.regression import AutoSklearnRegressor

In [54]:
auto_reg = AutoSklearnRegressor()
auto_reg.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py:691: RuntimeWarning:

overflow encountered in square



AutoSklearnRegressor(per_run_time_limit=360)

In [55]:
print_results(auto_reg)

MSE treino: 0.0011724120350682001
MSE validacao: 0.0025104949298226835
MSE teste: 0.0023729149877994627


# Diferença de MSE de treino = 0.0004687242670972402 - 0.0011724120350682001 = -0.00070368776797096

# Nosso modelo se saiu muito bem nos dados de treinamento, até melhor do que o auto-sklearn, com uma diferença muito pequena, então não há **bias**, descartando **underfitting**.

# Diferença de MSE de validação = 0.0030516863973971357-0.0025104949298226835= 0.0005411914675744521

# A proporção da diferença dos resultados no conjunto de validação se manteve, se mostrando até menor que a do conjunto de treinamento(baixa **variância**). Porém, como dessa vez nosso modelo se saiu um pouco pior do que o auto-sklearn, talvez seja indicativo de **overfitting**.

# **Melhorando o Modelo**

Como nosso erro no conjunto de validação e de teste é extremamente pequeno, provavelmente apenas aumentar o tamanho do dataset de treino corrija o problema

In [60]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=2020)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=2020)

In [61]:
randomforestreg = RandomForestRegressor(n_estimators=915, max_depth=30,min_samples_leaf=1)
randomforestreg.fit(X_train, y_train)

RandomForestRegressor(max_depth=30, n_estimators=915)

In [63]:
print_results(randomforestreg)

MSE treino: 0.0004487969474220763
MSE validacao: 0.0030810171188915287
MSE teste: 0.0027357395289580033


Estranhamente, o erro no conjunto de validação aumentou ligeiramente, mas diminuiu no conjunto de teste. Acreditamos que mesmo assim a proporção da diferença dos resultados do nosso modelo para o auto-sklearn são praticamente irrelevantes (da ordem de 10^-4), considerando assim ser um bom modelo.

# **Ngrok para visualizar mlflow**

In [64]:
get_ipython().system_raw("mlflow ui --port 5000 &")
from pyngrok import ngrok
ngrok.kill()
NGROK_AUTH_TOKEN = ""
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)

MLflow Tracking UI: https://22e79722d2e3.ngrok.io
